In [ ]:
!rm -rf AtomML-Course
!git clone https://github.com/AMLS-PRG/AtomML-Course

Cloning into 'AtomML-Course'...
remote: Enumerating objects: 2751, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 2751 (delta 87), reused 21 (delta 21), pack-reused 2625 (from 1)
Receiving objects: 100% (2751/2751), 47.27 MiB | 20.98 MiB/s, done.
Resolving deltas: 100% (1356/1356), done.


In [ ]:
pip install ase numpy matplotlib
import numpy as np
import matplotlib.pyplot as plt
from ase.io.lammpsrun import read_lammps_dump

In [ ]:
# ----------- File paths -----------
file_liquid = "AtomML-Course/module-5/04-Performing-MD-simulations/runLammps/md.dump"
file_solid  = "AtomML-Course/module-5/04-Performing-MD-simulations/runLammps/md_solid.dump"

# ----------- Parameters -----------
n_frames = 700          # last 700 frames
bin_width = 0.5         # Å
atom_type = 1           # target atom type

In [ ]:
# ----------- Density calculation function -----------

def compute_z_density(frames, bin_width, atom_type):
    """
    Compute the average atomic number density along the z-axis (in #/nm³).

    Parameters:
        frames: list of ASE Atoms objects
        bin_width: bin width in Å
        atom_type: atom type to include (int)

    Returns:
        z_centers: array of bin center positions (Å)
        avg_density: average density values over frames (#/nm³)
    """
    densities_all = []

    for atoms in frames:
        # Get box dimensions
        cell = atoms.get_cell()
        z_length = cell[2, 2]
        x_len = cell[0, 0]
        y_len = cell[1, 1]

        # Atom positions for given type
        z_pos = atoms.positions[atoms.arrays['type'] == atom_type][:, 2]

        # Histogram
        n_bins = int(np.ceil(z_length / bin_width))
        hist, bin_edges = np.histogram(z_pos, bins=n_bins, range=(0, z_length))

        # Bin volume in nm³
        bin_volume_nm3 = (x_len * y_len * bin_width) * 1e-3
        density = hist / bin_volume_nm3  # #/nm³
        densities_all.append(density)

    avg_density = np.mean(densities_all, axis=0)
    z_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

    return z_centers, avg_density


In [ ]:
# ----------- Load last 700 frames -----------

frames_liq = read_lammps_dump(file_liquid, index=slice(-n_frames, None))
frames_sol = read_lammps_dump(file_solid, index=slice(-n_frames, None))

# ----------- Compute densities -----------

z_liq, dens_liq = compute_z_density(frames_liq, bin_width, atom_type)
z_sol, dens_sol = compute_z_density(frames_sol, bin_width, atom_type)

In [ ]:
# ----------- Plot both densities -----------

plt.figure(figsize=(8, 5))
plt.plot(z_liq, dens_liq, label='Liquid', linestyle='--', marker='o', markersize=4)
plt.plot(z_sol, dens_sol, label='Solid', linestyle='--', marker='s', markersize=4)

plt.xlabel("z (Å)", fontsize=12)
plt.ylabel("Density (#/nm³)", fontsize=12)
plt.title("Z-axis Atomic Density Profile", fontsize=14)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("z_density_liquid_vs_solid.png")
plt.show()